In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
resnet = ResNet50(weights='imagenet', input_shape=(224, 224, 3))

In [6]:
img = tf.keras.preprocessing.image.load_img(
    "C:/Users/luke_/Desktop/Dev/ML/ee544-computer-vision/data/imagewoof-320/train/n02086240/n02086240_2.JPEG", target_size=(224, 224))
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

prediction = resnet.predict(img_array)
score = tf.nn.softmax(prediction[0])

In [7]:
score

<tf.Tensor: shape=(1000,), dtype=float32, numpy=
array([0.00099871, 0.00099871, 0.00099871, 0.00099871, 0.00099871,
       0.00099871, 0.00099871, 0.00099871, 0.00099871, 0.00099871,
       0.00099871, 0.00099871, 0.00099871, 0.00099871, 0.00099871,
       0.00099871, 0.00099871, 0.00099871, 0.00099871, 0.00099871,
       0.00099871, 0.00099871, 0.00099871, 0.00099871, 0.00099871,
       0.00099871, 0.00099871, 0.00099871, 0.00099871, 0.00099871,
       0.00099871, 0.00099871, 0.00099871, 0.00099871, 0.00099871,
       0.00099871, 0.00099871, 0.00099871, 0.00099871, 0.00099871,
       0.00099871, 0.00099871, 0.00099871, 0.00099871, 0.00099871,
       0.00099871, 0.00099871, 0.00099871, 0.00099871, 0.00099871,
       0.00099871, 0.00099871, 0.00099871, 0.00099871, 0.00099871,
       0.00099871, 0.00099871, 0.00099871, 0.00099871, 0.00099871,
       0.00099871, 0.00099871, 0.00099871, 0.00099871, 0.00099871,
       0.00099871, 0.00099871, 0.00099871, 0.00099871, 0.00099871,
       0.0009

In [6]:

labels_path = tf.keras.utils.get_file('ImageNetLabels.txt','https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
imagenet_labels = np.array(open(labels_path).read().splitlines())

In [9]:
predicted_class = np.argmax(prediction[0], axis=-1)
predicted_class

155

In [10]:
imagenet_labels[predicted_class]

'Pekinese'

In [7]:
def get_resnet50_imagenet_prediction(img_path):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)

    prediction = resnet.predict(img_array)
    return np.argmax(prediction[0], axis=-1)

In [10]:
import os
from skimage import io
# update folder names manually, checking files with imagenet weights out of curiosity to see if they match while I gather shape info and file counts
project_fldr = "C:/Users/luke_/Desktop/Dev/ee544-computer-vision"
data_fldr = f"{project_fldr}/data/imagewoof-320"
classes = ["australian terrier", "beagle", "border terrier", "dingo", "english foxhound", "golden retriever",
           "old english sheepdog", "rhodesian ridgeback", "samoyed", "shih tzu"]
results = {}
for c in classes:
    class_result = {}
    for fldr in ["train", "val"]:
        print(f"processing {c}-{fldr}...")
        curr_fldr = f"{data_fldr}/{fldr}/{c}"
        filenames = [f for f in os.listdir(curr_fldr) if os.path.isfile(os.path.join(curr_fldr, f))]
        img_shapes = []
        predictions = []
        for f in filenames:
            img_path = os.path.join(curr_fldr, f)
            img = io.imread(img_path)
            img_shapes.append(img.shape)
            predictions.append(get_resnet50_imagenet_prediction(img_path))
        class_result[fldr] = {"count": len(filenames), "shapes": img_shapes, "predictions": predictions}
        print(f"{c}{fldr}: {len(filenames)} images, min-shape={np.min(img_shapes)}, max-shape={np.max(img_shapes)}, "
              f"most common shape={np.argmax(img_shapes)}, most commonly predicted imagenet class={imagenet_labels[np.argmax(predictions)]}")
    results[c] = class_result
results

processing australian terrier-train...
australian terriertrain: 1300 images, min-shape=(320, 320, 3), max-shape=(629, 320, 3), most common shape=416, most commonly predicted imagenet class=burrito
processing australian terrier-val...
australian terrierval: 50 images, min-shape=3, max-shape=506, most common shape=49, most commonly predicted imagenet class=African chameleon
processing beagle-train...
beagletrain: 1300 images, min-shape=(320, 320, 3), max-shape=(594, 320, 3), most common shape=338, most commonly predicted imagenet class=recreational vehicle
processing beagle-val...
beagleval: 50 images, min-shape=3, max-shape=426, most common shape=1, most commonly predicted imagenet class=jay
processing border terrier-train...
border terriertrain: 1300 images, min-shape=(320, 320, 3), max-shape=(574, 320, 3), most common shape=785, most commonly predicted imagenet class=motor scooter
processing border terrier-val...
border terrierval: 50 images, min-shape=3, max-shape=480, most common sh

C:\Users\luke_\Anaconda3\envs\ee544-computer-vision\lib\site-packages\numpy\core\fromnumeric.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\luke_\Anaconda3\envs\ee544-computer-vision\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


{'australian terrier': {'train': {'count': 1300,
   'shapes': [(320, 426, 3),
    (320, 358, 3),
    (480, 320, 3),
    (480, 320, 3),
    (320, 320, 3),
    (320, 569, 3),
    (320, 384),
    (320, 327, 3),
    (320, 426, 3),
    (320, 400, 3),
    (320, 480, 3),
    (320, 352, 3),
    (320, 426, 3),
    (320, 463, 3),
    (320, 320, 3),
    (376, 320, 3),
    (483, 320, 3),
    (320, 426, 3),
    (480, 320, 3),
    (320, 426, 3),
    (320, 535, 3),
    (320, 449, 3),
    (320, 477, 3),
    (320, 426, 3),
    (320, 426, 3),
    (320, 480, 3),
    (320, 426, 3),
    (320, 426, 3),
    (320, 466, 3),
    (320, 492, 3),
    (320, 320, 3),
    (320, 336, 3),
    (320, 426, 3),
    (320, 426, 3),
    (425, 320, 3),
    (320, 404, 3),
    (463, 320, 3),
    (480, 320, 3),
    (320, 400, 3),
    (320, 480, 3),
    (476, 320, 3),
    (320, 426, 3),
    (320, 366, 3),
    (320, 334, 3),
    (597, 320, 3),
    (320, 320, 3),
    (320, 320, 3),
    (320, 426, 3),
    (320, 427, 3),
    (320, 481

In [11]:
results


{'australian terrier': {'train': {'count': 1300,
   'shapes': [(320, 426, 3),
    (320, 358, 3),
    (480, 320, 3),
    (480, 320, 3),
    (320, 320, 3),
    (320, 569, 3),
    (320, 384),
    (320, 327, 3),
    (320, 426, 3),
    (320, 400, 3),
    (320, 480, 3),
    (320, 352, 3),
    (320, 426, 3),
    (320, 463, 3),
    (320, 320, 3),
    (376, 320, 3),
    (483, 320, 3),
    (320, 426, 3),
    (480, 320, 3),
    (320, 426, 3),
    (320, 535, 3),
    (320, 449, 3),
    (320, 477, 3),
    (320, 426, 3),
    (320, 426, 3),
    (320, 480, 3),
    (320, 426, 3),
    (320, 426, 3),
    (320, 466, 3),
    (320, 492, 3),
    (320, 320, 3),
    (320, 336, 3),
    (320, 426, 3),
    (320, 426, 3),
    (425, 320, 3),
    (320, 404, 3),
    (463, 320, 3),
    (480, 320, 3),
    (320, 400, 3),
    (320, 480, 3),
    (476, 320, 3),
    (320, 426, 3),
    (320, 366, 3),
    (320, 334, 3),
    (597, 320, 3),
    (320, 320, 3),
    (320, 320, 3),
    (320, 426, 3),
    (320, 427, 3),
    (320, 481

In [12]:
from collections import Counter
shape_counter = Counter()
for c in results.keys():
    train_count = results[c]["train"]["count"]
    val_count = results[c]["val"]["count"]
    class_prediction_counter = Counter()
    for fldr in ["train", "val"]:
        class_prediction_counter.update(results[c][fldr]["predictions"])
        shape_counter.update(results[c][fldr]["shapes"])
    print(f"{c}: {train_count+val_count} imgs, {train_count} train, {val_count} val, most common prediction={class_prediction_counter.most_common(1)}")


australian terrier: 1350 imgs, 1300 train, 50 val, most common prediction=[('Airedale', 278)]
beagle: 1350 imgs, 1300 train, 50 val, most common prediction=[('basset', 305)]
border terrier: 1350 imgs, 1300 train, 50 val, most common prediction=[('Bedlington terrier', 270)]
dingo: 1350 imgs, 1300 train, 50 val, most common prediction=[('timber wolf', 115)]
english foxhound: 804 imgs, 754 train, 50 val, most common prediction=[('Walker hound', 149)]
golden retriever: 1350 imgs, 1300 train, 50 val, most common prediction=[('vizsla', 264)]
old english sheepdog: 1350 imgs, 1300 train, 50 val, most common prediction=[('komondor', 713)]
rhodesian ridgeback: 1350 imgs, 1300 train, 50 val, most common prediction=[('Scottish deerhound', 549)]
samoyed: 1350 imgs, 1300 train, 50 val, most common prediction=[('Great Pyrenees', 934)]
shih tzu: 1350 imgs, 1300 train, 50 val, most common prediction=[('Pekinese', 658)]


this shows the english foxhound class is under-represented. Will need to account for this